In [1]:
# update rmse 並記得重新啟動 Anaconda
# !pip install -U scikit-learn 

In [2]:
# Modules
import random
import pandas as pd
import numpy as np
from keras import models
from keras import layers
from keras import optimizers, metrics
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.layers.advanced_activations import LeakyReLU
import math
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from scipy.stats import norm, skew #for some statistics
from sklearn.preprocessing import PowerTransformer

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import r2_score
from keras import backend as K

import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [3]:

def lr_rmse_ave(x,y):
    
    
    train_rmse=[]
    test_rmse=[]
    test_r2=[]
    
    for i in np.arange(10):
        x_train,x_test,y_train,y_test = train_test_split(x,y, random_state = i)
        lr = LinearRegression().fit(x_train,y_train)

        y_train_pred = lr.predict(x_train)
        y_test_pred = lr.predict(x_test)
        train_rmse.append(mean_squared_error(y_train, y_train_pred,squared=False))
        test_rmse.append(mean_squared_error(y_test, y_test_pred,squared=False))
        test_r2.append(r2_score(y_test,y_test_pred))
    
    train_rmse=np.array(train_rmse).mean()
    test_rmse=np.array(test_rmse).mean()
    test_r2=np.array(test_r2).mean()
    
#     print('train_rmse:', train_rmse)
#     print('test_rmse:', test_rmse)
#     print('test_r2:', test_r2)
    return train_rmse, test_rmse, test_r2

In [4]:
def skew_pro(data):
    columns=data.drop(['Predict'], axis=1).columns
    for col in columns: 
        if abs(data[col].skew()) >= 0.7: 
            pt = PowerTransformer() 
            d = pt.fit_transform(data[col].values.reshape(-1, 1)).flatten()
            data[col]=d
    X = data[columns]
    scaler = RobustScaler()
    data[columns] = scaler.fit_transform(X)
    
    return data

In [5]:
def r2_keras(y_true, y_pred):
    SS_res =  K.sum(K.square(y_true - y_pred)) 
    SS_tot = K.sum(K.square(y_true - K.mean(y_true))) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

In [6]:
def nn_ave(x,y):
    
    rmse=[]
    r2=[]

    for i in np.arange(10):
        X_train,X_val,Y_train,Y_val  = train_test_split(x, y, test_size=0.2, random_state=i)
        
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        batch_size = 32
        epochs = 1000
        model = build_model()
        history = model.fit(
            X_train,
            Y_train,
            epochs = epochs,
            validation_data = (X_val, Y_val),
            batch_size = batch_size,
            shuffle = True,
            verbose=0
            )
        
        y_pred = model.predict(X_val)
        rmse.append(math.sqrt(mean_squared_error(Y_val , y_pred )))
        r2.append(r2_score(Y_val , y_pred ))
    
    test_rmse=np.array(rmse).mean()
    print('test_rmse_ave:',test_rmse)
    print(rmse)
    print('\n')
    test_r2=np.array(r2).mean()
    print('test_r2_ave:',test_r2)
    print(r2)

In [7]:
## 
n='Input_A3_013' # 改名稱
##
dire= ['Data/BayesianRidge_Pre_0/','Data/BayesianRidge_Pre_1/','Data/BayesianRidge_Pre_2/','Data/BayesianRidge_Pre_3/','Data/BayesianRidge_Pre_4/','Data/BayesianRidge_Pre_5/','Data/BayesianRidge_Pre_6/','Data/BayesianRidge_Pre_7/']
print(n)
for d in dire:
    f_train_rmse=[]
    f_test_rmse=[]
    f_test_r2=[]
    print(d)
    data_d= d+n+'.csv'
    data= pd.read_csv(data_d,index_col=0).drop(['Number'], axis=1)
    y=data.Predict
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    f20=abs(numcorr['Predict']).sort_values(ascending=False).head(21).to_frame().index.to_numpy()[1:]
    fea_list=[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20]
    for i in fea_list:
        print('feature number:',i)
        x=skew_data[f20[:i]]
        tr_r,te_r, te_r2= lr_rmse_ave(x,y)
        print('train_rmse:',tr_r)
        print('test_rmse:',te_r)
        print('test_r2:',te_r2)
        f_train_rmse.append(tr_r)
        f_test_rmse.append(te_r)
        f_test_r2.append(te_r2)

    test_r=np.array(f_test_rmse)
    test_r2=np.array(f_test_r2)
    test_r_min= np.where(test_r == np.amin(test_r))
    test_r2_max= np.where(test_r2 == np.amax(test_r2))    
    print('Test_rmse_min = 1.fea_num:', fea_list[test_r_min[0][0]], ' 2.rmse:' ,np.amin(test_r))
    print('Test_r2_max = 1. fea_num:', fea_list[test_r2_max[0][0]],  ' 2.r2:', np.amax(test_r2))

Input_A3_013
Data/BayesianRidge_Pre_0/
feature number: 2
train_rmse: 0.0014633161034438584
test_rmse: 0.0014723010657973603
test_r2: 0.19857732099877837
feature number: 3
train_rmse: 0.0014326004925003157
test_rmse: 0.001451349065164711
test_r2: 0.2174054674919276
feature number: 4
train_rmse: 0.0014027927656574996
test_rmse: 0.0014317717871205956
test_r2: 0.23819332642463795
feature number: 5
train_rmse: 0.001390545513971083
test_rmse: 0.0014277455515734308
test_r2: 0.24346557712638384
feature number: 6
train_rmse: 0.0013798295340172293
test_rmse: 0.0014260865268315675
test_r2: 0.24349209285691104
feature number: 7
train_rmse: 0.0013785975510688451
test_rmse: 0.0014313350542259125
test_r2: 0.23624049440115674
feature number: 8
train_rmse: 0.0013780261523848069
test_rmse: 0.001433716921943052
test_r2: 0.23364016483690322
feature number: 9
train_rmse: 0.0013611312986392334
test_rmse: 0.0014351503101384623
test_r2: 0.2322770504704282
feature number: 10
train_rmse: 0.0013589010171208367
t

train_rmse: 0.001334737445158464
test_rmse: 0.0014464895745527429
test_r2: 0.21851760673228743
feature number: 16
train_rmse: 0.001332050308863564
test_rmse: 0.0014576101222509982
test_r2: 0.20444492515355517
feature number: 17
train_rmse: 0.001330108311079849
test_rmse: 0.0014679045302612031
test_r2: 0.1906583304676595
feature number: 18
train_rmse: 0.0013285749183062536
test_rmse: 0.0014758690293844886
test_r2: 0.18245337586347207
feature number: 19
train_rmse: 0.0013196426425039571
test_rmse: 0.0014670977883806985
test_r2: 0.19414493473040434
feature number: 20
train_rmse: 0.0013132754407153063
test_rmse: 0.0014673958417874707
test_r2: 0.1902564183085314
Test_rmse_min = 1.fea_num: 7  2.rmse: 0.0014037982867130328
Test_r2_max = 1. fea_num: 7  2.r2: 0.26721159879824874
Data/BayesianRidge_Pre_4/
feature number: 2
train_rmse: 0.0014633161034438584
test_rmse: 0.0014723010657973603
test_r2: 0.19857732099877837
feature number: 3
train_rmse: 0.0014326004925003157
test_rmse: 0.00145134906516

train_rmse: 0.001366182884562912
test_rmse: 0.0014037982867130328
test_r2: 0.26721159879824874
feature number: 8
train_rmse: 0.0013645056088834975
test_rmse: 0.0014084771637673276
test_r2: 0.2608325599114588
feature number: 9
train_rmse: 0.0013640531987973742
test_rmse: 0.0014111525350296791
test_r2: 0.25792091004022577
feature number: 10
train_rmse: 0.0013573164116204663
test_rmse: 0.0014129661249084541
test_r2: 0.25610472775184545
feature number: 11
train_rmse: 0.0013411323869744127
test_rmse: 0.0014197755458984665
test_r2: 0.2494737139618159
feature number: 12
train_rmse: 0.001339190660863195
test_rmse: 0.0014205134613991754
test_r2: 0.24924753878476585
feature number: 13
train_rmse: 0.0013371382144589795
test_rmse: 0.0014294536809593722
test_r2: 0.2381185020605403
feature number: 14
train_rmse: 0.0013353655672052104
test_rmse: 0.0014436263549107956
test_r2: 0.2216241222010495
feature number: 15
train_rmse: 0.001334737445158464
test_rmse: 0.0014464895745527429
test_r2: 0.21851760673

# 可以修改第一層的神經元數量

In [8]:
#
def build_model():
    model = models.Sequential()
#
    model.add(layers.Dense(6, activation='relu',input_shape=(x.shape[1],))) # 調這裡layers的數量
#
    model.add(layers.Dense(1, activation='linear'))
    model.compile(optimizer='adam', loss=metrics.mean_squared_error, metrics=[r2_keras])

    return model

In [20]:
fea_num=[5,6,7,8,9,10] # 要測試的feature數量 神經元 6

for fea in fea_num:
    
    print('feature number:',fea)
    #
    data=pd.read_csv('Data/BayesianRidge_Pre_2/Input_A3_013.csv',index_col=0).drop(['Number'], axis=1) ## 改路徑!!!
    #
    skew_data=skew_pro(data)
    num=skew_data.select_dtypes(exclude='object')
    numcorr=num.corr()
    title=numcorr['Predict'].sort_values(ascending=False).head(fea+1).to_frame().index.to_numpy()[1:]
    x = skew_data[title]
    y = skew_data.Predict
    nn_ave(x,y)
    print('\n')

feature number: 5
test_rmse_ave: 0.0018982184578050245
[0.0024599099450230435, 0.0020810529299277055, 0.0019291239233064218, 0.0015698901723207273, 0.001430020257416422, 0.0017522276598014171, 0.0024766943508015833, 0.0019425634956926204, 0.0014338218659043717, 0.0019068799778559347]


test_r2_ave: -0.3385532434258224
[-1.5943362493966253, -0.23348223413667513, -0.18105204962554966, -0.4684241549648709, 0.04422988463901989, -0.0882869416730645, -0.8673380371355073, -0.15883738910177447, 0.16856708472986792, -0.006572347593043704]


feature number: 6
test_rmse_ave: 0.002462631254138334
[0.0018134463378661617, 0.0015683144613624035, 0.0016011191340830936, 0.0013455141476450465, 0.0012874459753181697, 0.004434249318235345, 0.0035589758168359615, 0.00173134411976543, 0.0016180367110276736, 0.005667866519244056]


test_r2_ave: -1.647494887727619
[-0.4099290698721625, 0.2994599963020006, 0.18642783941176977, -0.07867227327499804, 0.22531152395283593, -5.969515218440833, -2.8559215604891692, 